# Выбор локации для скважины

«ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Условия задачи:
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.


Необходимо:

- Для обучения модели использовать только линейную регрессия
- Предоставить средний запас предсказанного сырья и RMSE модели в каждом регионе
- Расчитать достаточный объём сырья для безубыточной разработки новой скважины
- Анализировать возможную прибыль и риски техникой *Bootstrap.*

In [1]:
import warnings
warnings.simplefilter('ignore')

## Загрузка и подготовка данных

In [2]:
#import sklearn

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

Согласно предоставленной документации
Данные геологоразведки трёх регионов находятся в файлах

In [3]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
df0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


В которых:  
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).  

In [5]:
print(f'Количество уникальных значенией в geo_data_0 : {df0.id.nunique()} из {df0.shape[0]} записей')
print(f'Количество уникальных значенией в geo_data_1 : {df1.id.nunique()} из {df1.shape[0]} записей')
print(f'Количество уникальных значенией в geo_data_2 : {df2.id.nunique()} из {df2.shape[0]} записей')

Количество уникальных значенией в geo_data_0 : 99990 из 100000 записей
Количество уникальных значенией в geo_data_1 : 99996 из 100000 записей
Количество уникальных значенией в geo_data_2 : 99996 из 100000 записей


В данных имееется несколько записей с разными данными признаков для скважин с одинаковыми `id`.    
Удалим их из дальнейшего анализа

In [6]:
def preproc_df(df):
  dupl_id = df.groupby(df.id, as_index=False).size()\
             .sort_values(by='size', ascending=False)\
             .query('size>1')\
             ['id'].values

  df = df[~df.id.isin(dupl_id)]

  df = df.drop('id', axis=1)
  
  return df

In [7]:
df0 = preproc_df(df0)
df1 = preproc_df(df1)
df2 = preproc_df(df2)

## Обучение и проверка модели

In [8]:
def modeling(df):
  X_train, X_test, y_train, y_test = train_test_split(df.drop('product', axis=1),
                                                    df['product'],
                                                    test_size=0.25,
                                                    random_state=42)
  lr = LinearRegression(normalize=True)
  lr.fit(X_train,y_train)
  print(f"Средний запас сырья: {df['product'].mean()}")
  print(f"RMSE = {mean_squared_error(y_test, lr.predict(X_test))**0.5}")
  predicted_test = pd.concat([X_test,
                  y_test,
                  pd.DataFrame(lr.predict(X_test),
                        columns=['predict'],
                        index=X_test.index)],
                  axis=1)
  return predicted_test

In [9]:
predictions = [df0,df1,df2]
for i, df in enumerate (predictions):
  print(f'geolocation {i}')
  predictions[i] = modeling(df)
  print()

geolocation 0
Средний запас сырья: 92.49916597893444
RMSE = 37.76114714593895

geolocation 1
Средний запас сырья: 68.82414772665173
RMSE = 0.8928635898777318

geolocation 2
Средний запас сырья: 94.99876686768079
RMSE = 40.131155604130505



## Подготовка к расчёту прибыли

In [10]:
budget = 10_000_000_000
wells = 500
best_wells = 200
price = 450_000
limit = 0.025

In [11]:
print(f'Достаточный объём сырья для безубыточной разработки новой скважины : {budget/best_wells/price:.2f} тыс. баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины : 111.11 тыс. баррелей


Функция для подсчета прибыли для лучших предсказаний

In [12]:
def predicter(df):
  return (df.sort_values(by="predict", ascending=False)\
          ["product"][:best_wells].sum()*price-budget)/1_000_000

## Расчёт прибыли и рисков 

In [13]:
for i, df in enumerate(predictions):
  print(f'''Прибыль по скважинам с наилучшим прогнозом в {i} области :
   {predicter(predictions[i]):.2f} млн.''')  
  print()

Прибыль по скважинам с наилучшим прогнозом в 0 области :
   3301.82 млн.

Прибыль по скважинам с наилучшим прогнозом в 1 области :
   2415.09 млн.

Прибыль по скважинам с наилучшим прогнозом в 2 области :
   2103.91 млн.



In [14]:
def get_bootstrap(data, index,
                  boot_it=1000,
                  boot_len=500,
                  bootstrap_conf_level=0.95,
                  satisfaction='не удовлетворительный',
                  state = np.random.RandomState(42)):
  boot_data = []
  for i in range(boot_it):
    samples = data.sample(boot_len, random_state=state)
    boot_data.append(predicter(samples))
  boot_data = pd.Series(boot_data)
  print(f'Средняя прибыль региона {index}: {boot_data.mean():.2f} млн.')
  left_quant = (1 - bootstrap_conf_level)/2
  right_quant = 1 - (1 - bootstrap_conf_level)/2
  print(f'95%-й доверительный интервал: {boot_data.quantile([left_quant, right_quant]).values}')
  risk = (boot_data<0).mean()
  if risk<limit:
    satisfaction='удовлетворительный'
  print(f'Риск убытков {satisfaction}, так как равен {risk*100:.1f}%')
  print()


In [15]:
for i, df in enumerate(predictions):
  get_bootstrap(df, i)

Средняя прибыль региона 0: 402.55 млн.
95%-й доверительный интервал: [-106.97801294  934.99722656]
Риск убытков не удовлетворительный, так как равен 5.7%

Средняя прибыль региона 1: 487.08 млн.
95%-й доверительный интервал: [ 94.50733161 897.96123635]
Риск убытков удовлетворительный, так как равен 0.9%

Средняя прибыль региона 2: 359.53 млн.
95%-й доверительный интервал: [-171.58925319  841.4618113 ]
Риск убытков не удовлетворительный, так как равен 8.3%



## Выводы

1. Регион 1 самый прибыльный и наименее рискованный для освоения.
2. Данные для проекта и условия невероятно унылы.